# Task B - single subject - model ?

## Notebook setup
This first cell contains the parameters that can be tuned for code execution:
- subject: select the subject on which to test the model, between [1,4];
- label: index of feature column to be selected to perform activity detection, between [0,6]. The default value for task B is 6;
- folder: directory name where '.mat' files are stored;
- window_size: parameter that sets the length of temporal windows on which to perform the convolution;
- stride: step length to chose the next window.

In [1]:
subject = 2
label = 6
folder = "../data/full/"
window_size = 15
stride = 5

In [2]:
import preprocessing
import models
import numpy as np
from sklearn.metrics import classification_report
from keras.models import load_model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical

C:\Users\Riccardo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In the following cell, we make use of some functions of Keras which have been removed, but of which the code is still available at https://github.com/keras-team/keras/commit/a56b1a55182acf061b1eb2e2c86b48193a0e88f7. These are used to evaulate the f1 score during training on batches of data: this is only an approximation though, which is the reason why they have been removed.

In [3]:
import os
if not(os.path.exists("./data")):
    os.mkdir("./data")

# One-shot classification
Here classification is performed with null class.
### Preprocessing

In [4]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=True,
                                                                                                print_info=False)

Y_train and Y_test contain the correct labels for each signals window. Y_test in particular will be used to evaluate predictions for both this (one-shot) and the two-steps models. For this reason it is here saved with a different name, to avoid having it being overwritten later.

In [5]:
Y_test_true = Y_test

### Model

In [6]:
oneshot_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

oneshot_model.compile(optimizer = Adam(lr=0.001),
                      loss = "categorical_crossentropy", 
                      metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BOS_2.hdf5', verbose=1, save_best_only=True)

### Training

In [7]:
X_train.shape

(29159, 15, 110)

In [8]:
oneshot_model.fit(x = X_train, 
                  y = to_categorical(Y_train),
                  epochs = 15,
                  batch_size = 16,
                  verbose = 1,
                  callbacks=[checkpointer],
                  validation_data=(X_test, to_categorical(Y_test)),
                  class_weight=class_weights)

Train on 29159 samples, validate on 11542 samples
Epoch 1/15
29159/29159 [==============================] - 62s 2ms/step - loss: 1.2292 - acc: 0.6530 - val_loss: 0.6553 - val_acc: 0.8259

Epoch 00001: val_loss improved from inf to 0.65528, saving model to ./data/model_BOS_2.hdf5
Epoch 2/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.9829 - acc: 0.6915 - val_loss: 0.7193 - val_acc: 0.8067

Epoch 00002: val_loss did not improve
Epoch 3/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.8749 - acc: 0.7167 - val_loss: 0.6991 - val_acc: 0.8272

Epoch 00003: val_loss did not improve
Epoch 4/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.7943 - acc: 0.7372 - val_loss: 0.7343 - val_acc: 0.8029

Epoch 00004: val_loss did not improve
Epoch 5/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.7386 - acc: 0.7531 - val_loss: 0.7230 - val_acc: 0.8055

Epoch 00005: val_loss did not improve
Epoch 6/15
29159

### Evaluation - passare class_weights a class report

In [9]:
Y_pred = oneshot_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.92      0.91      0.91      9699
          1       0.20      0.14      0.16       108
          2       0.03      0.01      0.02        76
          3       0.00      0.00      0.00        34
          4       0.06      0.08      0.07        51
          5       0.25      0.34      0.29       109
          6       0.01      0.03      0.02        35
          7       0.44      0.44      0.44       212
          8       0.11      0.12      0.11       119
          9       0.17      0.16      0.16       141
         10       0.00      0.00      0.00        74
         11       0.00      0.00      0.00        47
         12       0.07      0.24      0.11        62
         13       0.18      0.38      0.24       110
         14       0.07      0.10      0.08        51
         15       0.37      0.29      0.32       262
         16       0.10      0.04      0.05       185
         17       0.07      0.07      0.07   

In [10]:
oneshot_model_best = load_model('./data/model_BOS_2.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

             precision    recall  f1-score   support

          0       0.86      0.97      0.91      9699
          1       0.00      0.00      0.00       108
          2       0.00      0.00      0.00        76
          3       0.00      0.00      0.00        34
          4       0.00      0.00      0.00        51
          5       0.30      0.03      0.05       109
          6       0.00      0.00      0.00        35
          7       0.45      0.33      0.38       212
          8       0.00      0.00      0.00       119
          9       0.18      0.06      0.09       141
         10       0.00      0.00      0.00        74
         11       0.50      0.02      0.04        47
         12       0.00      0.00      0.00        62
         13       0.16      0.11      0.13       110
         14       0.11      0.08      0.09        51
         15       0.19      0.14      0.16       262
         16       0.00      0.00      0.00       185
         17       0.00      0.00      0.00   

C:\Users\Riccardo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Two-steps classification
## Activity detection
This model performs a binary classification.
### Preprocessing

In [11]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=True,
                                                                                                null_class=True,
                                                                                                print_info=False)

### Model

In [ ]:
detection_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

detection_model.compile(optimizer = Adam(lr=0.001),
                        loss = "categorical_crossentropy", 
                        metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BTSD_2.hdf5', verbose=1, save_best_only=True)

### Training

In [ ]:
detection_model.fit(x = X_train, 
                    y = to_categorical(Y_train), 
                    epochs = 15, 
                    batch_size = 16,
                    verbose = 1,
                    callbacks=[checkpointer],
                    validation_data=(X_test, to_categorical(Y_test)),
                    class_weight=class_weights)

Train on 29159 samples, validate on 11542 samples
Epoch 1/15
29159/29159 [==============================] - 60s 2ms/step - loss: 0.5305 - acc: 0.7361 - val_loss: 0.3238 - val_acc: 0.8488

Epoch 00001: val_loss improved from inf to 0.32378, saving model to ./data/model_BTSD_2.hdf5
Epoch 2/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.4742 - acc: 0.7718 - val_loss: 0.3091 - val_acc: 0.8566

Epoch 00002: val_loss improved from 0.32378 to 0.30906, saving model to ./data/model_BTSD_2.hdf5
Epoch 3/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.4384 - acc: 0.7944 - val_loss: 0.3626 - val_acc: 0.8486

Epoch 00003: val_loss did not improve
Epoch 4/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.4132 - acc: 0.8135 - val_loss: 0.3595 - val_acc: 0.8248

Epoch 00004: val_loss did not improve
Epoch 5/15
29159/29159 [==============================] - 57s 2ms/step - loss: 0.3978 - acc: 0.8172 - val_loss: 0.3631 - val_acc: 0.

### Evaluation

In [ ]:
Y_pred = detection_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
detection_model_best = load_model('./data/model_BTSD_2.hdf5')

Y_pred = detection_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
Y_pred_d = Y_pred

## Activity classification

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=False,
                                                                                                print_info=False)

### Model

In [ ]:
classification_model = models.MotionDetection((window_size, n_features), n_classes, print_info=False)

classification_model.compile(optimizer = Adam(lr=0.001),
                             loss = "categorical_crossentropy", 
                             metrics = ["accuracy"])

checkpointer = ModelCheckpoint(filepath='./data/model_BTSC_2.hdf5', verbose=1, save_best_only=True)

### Training

In [ ]:
classification_model.fit(x = X_train,
                         y = to_categorical(Y_train), 
                         epochs = 15, 
                         batch_size = 16,
                         verbose = 1,
                         callbacks=[checkpointer],
                         validation_data=(X_test, to_categorical(Y_test)),
                         class_weight=class_weights)

### Evaluation

In [ ]:
Y_pred = classification_model.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

In [ ]:
classification_model_best = load_model('./data/model_BTSC_2.hdf5')

Y_pred = classification_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

## Cascade of detection and classification
The labels that have to be used for assessment are saved in Y_test_true. The labels predicted by the detection_model are saved instead in Y_pred_d.

In [ ]:
print(Y_test_true.shape, Y_pred_d.shape)

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=True,
                                                                                                null_class=True,
                                                                                                print_info=False)

In [ ]:
mask = (Y_pred_d == 1)
X_detected = X_test[mask, :, :]
Y_pred_c = classification_model_best.predict_classes(X_detected)
Y_pred_d[mask] = Y_pred_c

In [ ]:
print(classification_report(Y_test_true, Y_pred_d))

One-shot classification instead had:

In [ ]:
X_train, Y_train, X_test, Y_test, n_features, n_classes, class_weights = preprocessing.loadData(subject=subject,
                                                                                                label=label,
                                                                                                folder=folder,
                                                                                                window_size=window_size,
                                                                                                stride=stride,
                                                                                                make_binary=False,
                                                                                                null_class=True,
                                                                                                print_info=False)

In [ ]:
oneshot_model_best = load_model('./data/model_BOS_2.hdf5')

Y_pred = oneshot_model_best.predict_classes(X_test)
print(classification_report(Y_test, Y_pred))

# end